In [32]:
import numpy as np
import cv2

def skew(v):
    """
    Create a skew-symmetric matrix from a given vector v.
    """
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])


# Given camera matrix and extrinsics
K = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float)
R1 = cv2.Rodrigues(np.array([-1.6, 0.3, -2.1]))[0]
t1 = np.array([[0.0], [1.0], [3.0]], float)
R2 = cv2.Rodrigues(np.array([-0.4, -1.3, -1.6]))[0]
t2 = np.array([[0.0], [1.0], [6.0]], float)
R3 = cv2.Rodrigues(np.array([2.5, 1.7, -0.4]))[0]
t3 = np.array([[2.0], [-7.0], [25.0]], float)

# Observed points in each camera
p1 = np.array([[1046.0], [453.0]])
p2 = np.array([[1126.0], [671.0]])
p3 = np.array([[1165.0], [453.0]])


# Compute relative rotation and translation between camera 1 and camera 2
R_rel = np.dot(R2, R1.T)
t_rel = np.dot(R2, t1) + t2

# Construct the essential matrix E12 from the relative rotation and translation
E12 = skew(t_rel).dot(R_rel)

# Compute the fundamental matrix F12 from the essential matrix E12 and camera intrinsic matrices
F12 = np.dot(np.linalg.inv(K).T, np.dot(E12, np.linalg.inv(K)))

# Compute the epipolar line in camera 2 corresponding to p1
epipolar_line_2_homogeneous = np.dot(F12, p1_homogeneous)

# Extract coefficients of the epipolar line (ax + by + c = 0)
a, b, c = epipolar_line_2_homogeneous.flatten()

# Convert p2 from homogeneous to inhomogeneous coordinates
p2_inhomogeneous = p2_homogeneous[:2] / p2_homogeneous[2]

# Compute the distance from p2 to the epipolar line
distance = np.abs(a * p2_inhomogeneous[0] + b * p2_inhomogeneous[1] + c) / np.sqrt(a**2 + b**2)
print("Distance from p2 to the epipolar line in camera 2 corresponding to p1:", distance)

Distance from p2 to the epipolar line in camera 2 corresponding to p1: [340.34285878]


C:\Users\Nedan\AppData\Local\Temp\ipykernel_20952\2509943317.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[0, -v[2], v[1]],


In [39]:
# Print the computed essential matrix E12
print("Essential matrix E12:")
print(E12)

# Compute the fundamental matrix F12 from the essential matrix E12 and camera intrinsic matrices
F12 = np.dot(np.linalg.inv(K).T, np.dot(E12, np.linalg.inv(K)))
print("\nFundamental matrix F12:")
print(F12)

# Compute the epipolar line in camera 2 corresponding to p1
epipolar_line_2_homogeneous = np.dot(F12, p1_homogeneous)
print("\nEpipolar line in camera 2 corresponding to p1:")
print(epipolar_line_2_homogeneous)

# Extract coefficients of the epipolar line (ax + by + c = 0)
a, b, c = epipolar_line_2_homogeneous.flatten()
print("\nCoefficients of the epipolar line (ax + by + c = 0):")
print("a:", a, "b:", b, "c:", c)

# Convert p2 from homogeneous to inhomogeneous coordinates
p2_inhomogeneous = p2_homogeneous[:2] / p2_homogeneous[2]
print("\nHomogeneous p2:", p2_homogeneous)
print("Inhomogeneous p2:", p2_inhomogeneous)

# Compute the distance from p2 to the epipolar line
distance = np.abs(a * p2_inhomogeneous[0] + b * p2_inhomogeneous[1] + c) / np.sqrt(a**2 + b**2)
print("\nDistance from p2 to the epipolar line in camera 2 corresponding to p1:", distance)


Essential matrix E12:
[[array([-6.00717643]) array([-5.06311891]) array([3.23615186])]
 [array([-0.03142135]) array([-4.1530644]) array([-6.42721971])]
 [array([-0.0733929]) array([1.93059916]) array([3.15099781])]]

Fundamental matrix F12:
[[array([-7.4162672e-06]) array([-6.25076409e-06]) array([0.0153441])]
 [array([-3.87917857e-08]) array([-5.12724e-06]) array([-0.00397223])]
 [array([0.00787752]) array([0.01196104]) array([-12.06538513])]]

Epipolar line in camera 2 corresponding to p1:
[[array([0.00475508])]
 [array([-0.00633545])]
 [array([1.59285507])]]

Coefficients of the epipolar line (ax + by + c = 0):
a: [0.00475508] b: [-0.00633545] c: [1.59285507]

Homogeneous p2: [[1.126e+03]
 [6.710e+02]
 [1.000e+00]]
Inhomogeneous p2: [[1126.]
 [ 671.]]

Distance from p2 to the epipolar line in camera 2 corresponding to p1: [340.34285878]


In [33]:
p2_homogeneous

array([[1.126e+03],
       [6.710e+02],
       [1.000e+00]])

In [34]:
p2_inhomogeneous

array([[1126.],
       [ 671.]])

In [35]:
epipolar_line_2_homogeneous.flatten()

array([array([0.00475508]), array([-0.00633545]), array([1.59285507])],
      dtype=object)

In [36]:
a * p2_homogeneous[0] + b * p2_homogeneous[1] + c

array([2.6959951])

In [38]:
np.sqrt(a**2 + np.abs(b)**2)

array([0.00792141])

In [41]:
# Create a combined intrinsic matrix K_combined
K_combined = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float)

# Stacking the projection equations for all observations
# Stacking the projection equations for all observations
A = np.vstack([
    np.hstack([p1[0] * R1[2] - R1[0, 2] * np.eye(3), p1[1] * R1[2] - R1[1, 2] * np.eye(3)]),
    np.hstack([p2[0] * R2[2] - R2[0, 2] * np.eye(3), p2[1] * R2[2] - R2[1, 2] * np.eye(3)]),
    np.hstack([p3[0] * R3[2] - R3[0, 2] * np.eye(3), p3[1] * R3[2] - R3[1, 2] * np.eye(3)])
])

# Perform singular value decomposition (SVD) to solve for the 3D point
U, S, V = np.linalg.svd(A)
P_triangulated = V[-1, :4] / V[-1, 3]  # Normalize the homogeneous coordinates

# The triangulated point is the homogeneous 3D point
print("Triangulated point (homogeneous coordinates):", P_triangulated)


Triangulated point (homogeneous coordinates): [-0.65175314 -0.7280881  -0.36574491  1.        ]
